In [1]:
# Importing libraries
from pyspark.sql import SparkSession
import os

In [2]:
os.environ['JAVA_HOME'] = "C:\Program Files\Java\jdk-11"

In [3]:
#SparkSession
spark = SparkSession.builder \
    .appName("LinearRegressionExample") \
    .getOrCreate()

In [4]:
nasa = spark.read.csv("nasa.csv", header=True, inferSchema=True)

In [5]:
nasa.show(10)

+-----+-----+---+-------------------+-------+--------+-----+------------+--------+---------+------+------+---------+----------+-------+-------+----------+----------------+-------------+------+-----------+-----------+----+---------+---------+--------+--------+----------+----------+---------+--------+----------+---------+-------+---------+--------+---+--------+---------+--------+---------+---------+--------+---------+---------+----+----+-----+---+---+---+----+----+----+
| year|month|day|              td_ge|     dt|luna_num|saros|eclipse_type|   gamma|magnitude|lat_ge|lng_ge|lat_dd_ge| lng_dd_ge|sun_alt|sun_azm|path_width|central_duration|duration_secs|cat_no|canon_plate|julian_date|  t0|       x0|       x1|      x2|      x3|        y0|        y1|       y2|      y3|        d0|       d1|     d2|      mu0|     mu1|mu2|     l10|      l11|     l12|      l20|      l21|     l22|   tan_f1|   tan_f2|tmin|tmax|etype|PNS|UNS|NCN|nSer|nSeq|nJLE|
+-----+-----+---+-------------------+-------+--------+

In [6]:
nasa.tail(10)

[Row(year=2996, month=7, day=6, td_ge=datetime.datetime(2024, 4, 12, 23, 44, 3), dt=4395.4, luna_num=12325, saros=162, eclipse_type='A', gamma=0.5013, magnitude=0.9508, lat_ge='51.6N', lng_ge='145.6W', lat_dd_ge=51.58156, lng_dd_ge=-145.64313, sun_alt=59.7, sun_azm=198.9, path_width=208.3, central_duration='04m44s', duration_secs=284.4, cat_no=11889.0, canon_plate=595.0, julian_date=2815514.489, t0=0.0, x0=0.242102, x1=0.4963752, x2=-4.63e-05, x3=-5.63e-06, y0=0.459302, y1=-0.1119435, y2=-0.0001537, y3=1.4e-06, d0=22.294821, d1=-0.004573, d2=-5e-06, mu0=178.18694, mu1=14.99965, mu2=0.0, l10=0.564323, l11=4.12e-05, l12=-9.8e-06, l20=0.018095, l21=4.1e-05, l22=-9.7e-06, tan_f1=0.0046035, tan_f2=0.0045806, tmin=-3.0, tmax=3.0, etype=2, PNS=1, UNS=0, NCN=0, nSer=70, nSeq=42, nJLE=2),
 Row(year=2996, month=12, day=31, td_ge=datetime.datetime(2024, 4, 12, 12, 58, 17), dt=4399.0, luna_num=12331, saros=167, eclipse_type='T', gamma=-0.1729, magnitude=1.02488, lat_ge='32.9S', lng_ge='6.2E', lat_

In [7]:
num_rows = nasa.count()
num_columns = len(nasa.columns)
print("Shape of the DataFrame: ({}, {})".format(num_rows, num_columns))

Shape of the DataFrame: (11898, 54)


In [8]:
nasa.dtypes 

[('year', 'int'),
 ('month', 'int'),
 ('day', 'int'),
 ('td_ge', 'timestamp'),
 ('dt', 'double'),
 ('luna_num', 'int'),
 ('saros', 'int'),
 ('eclipse_type', 'string'),
 ('gamma', 'double'),
 ('magnitude', 'double'),
 ('lat_ge', 'string'),
 ('lng_ge', 'string'),
 ('lat_dd_ge', 'double'),
 ('lng_dd_ge', 'double'),
 ('sun_alt', 'double'),
 ('sun_azm', 'double'),
 ('path_width', 'double'),
 ('central_duration', 'string'),
 ('duration_secs', 'double'),
 ('cat_no', 'double'),
 ('canon_plate', 'double'),
 ('julian_date', 'double'),
 ('t0', 'double'),
 ('x0', 'double'),
 ('x1', 'double'),
 ('x2', 'double'),
 ('x3', 'double'),
 ('y0', 'double'),
 ('y1', 'double'),
 ('y2', 'double'),
 ('y3', 'double'),
 ('d0', 'double'),
 ('d1', 'double'),
 ('d2', 'double'),
 ('mu0', 'double'),
 ('mu1', 'double'),
 ('mu2', 'double'),
 ('l10', 'double'),
 ('l11', 'double'),
 ('l12', 'double'),
 ('l20', 'double'),
 ('l21', 'double'),
 ('l22', 'double'),
 ('tan_f1', 'double'),
 ('tan_f2', 'double'),
 ('tmin', 'doub

In [9]:
from pyspark.sql.functions import col, count
missing_values = nasa.agg(*[count(col(c)).alias(c) for c in nasa.columns])
missing_values.show()

+-----+-----+-----+-----+-----+--------+-----+------------+-----+---------+------+------+---------+---------+-------+-------+----------+----------------+-------------+------+-----------+-----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+
| year|month|  day|td_ge|   dt|luna_num|saros|eclipse_type|gamma|magnitude|lat_ge|lng_ge|lat_dd_ge|lng_dd_ge|sun_alt|sun_azm|path_width|central_duration|duration_secs|cat_no|canon_plate|julian_date|   t0|   x0|   x1|   x2|   x3|   y0|   y1|   y2|   y3|   d0|   d1|   d2|  mu0|  mu1|  mu2|  l10|  l11|  l12|  l20|  l21|  l22|tan_f1|tan_f2| tmin| tmax|etype|  PNS|  UNS|  NCN| nSer| nSeq| nJLE|
+-----+-----+-----+-----+-----+--------+-----+------------+-----+---------+------+------+---------+---------+-------+-------+----------+----------------+-------------+------+-----------+-----------+-----+-----+----

In [10]:
columns_to_drop = ['t0', 'x0', 'x1', 'x2', 'x3', 'y0', 'y1', 'y2', 'y3', 'd0', 
                   'd1', 'd2', 'mu0', 'mu1', 'mu2', 'l10', 'l11', 'l12', 'l20', 
                   'l21', 'l22', 'tan_f1', 'tan_f2', 'tmin', 'tmax', 'etype', 
                   'PNS', 'UNS', 'NCN', 'nSer', 'nSeq', 'nJLE']
nasa = nasa.drop(*columns_to_drop)

In [11]:
nasa.show()

+-----+-----+---+-------------------+-------+--------+-----+------------+--------+---------+------+------+---------+----------+-------+-------+----------+----------------+-------------+------+-----------+-----------+
| year|month|day|              td_ge|     dt|luna_num|saros|eclipse_type|   gamma|magnitude|lat_ge|lng_ge|lat_dd_ge| lng_dd_ge|sun_alt|sun_azm|path_width|central_duration|duration_secs|cat_no|canon_plate|julian_date|
+-----+-----+---+-------------------+-------+--------+-----+------------+--------+---------+------+------+---------+----------+-------+-------+----------+----------------+-------------+------+-----------+-----------+
|-1999|    6| 12|2024-04-12 03:14:51|46438.2|  -49456|    5|           T|-0.27009|  1.07329|  6.0N| 33.3W|  6.02745| -33.32629|   74.4|  343.7|     246.6|          06m37s|        397.2|   1.0|        1.0| 991085.635|
|-1999|   12|  5|2024-04-12 23:45:23|46426.5|  -49450|   10|           A|-0.23172|  0.93818| 32.9S| 10.8E|-32.92723|  10.79871|   76

In [12]:
from pyspark.sql.functions import mean, stddev, col, abs as pyspark_abs

numeric_columns = [col_name for col_name, col_type in nasa.dtypes if col_type != 'string' and col_type != 'timestamp']

stats = nasa.select(*(mean(col(c)).alias(c + '_mean') for c in numeric_columns),
                    *(stddev(col(c)).alias(c + '_std') for c in numeric_columns)).collect()[0]

threshold = 3

z_score_exprs = [
    ((col(c) - stats[c + '_mean']) / stats[c + '_std']).alias(c + '_z_score')
    for c in numeric_columns
]

nasa_with_z_scores = nasa.select(*z_score_exprs)

filter_conditions = [
    pyspark_abs(col(c + '_z_score')) <= threshold
    for c in numeric_columns
]

combined_filter_condition = filter_conditions[0]
for condition in filter_conditions[1:]:
    combined_filter_condition = combined_filter_condition & condition

outliers_removed_nasa = nasa_with_z_scores.filter(
    combined_filter_condition
).drop(*[c + '_z_score' for c in numeric_columns])

outliers_removed_nasa.show()

++
||
++
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
++
only showing top 20 rows



In [13]:
num_rows = nasa.count()
num_columns = len(nasa.columns)
print("Shape of the DataFrame: ({}, {})".format(num_rows, num_columns))

Shape of the DataFrame: (11898, 22)
